In [1]:

import sys
sys.path.insert(0, '/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3_simula/Handwriting-1-master/VerticalAttentionOCR/')
from torch.optim import Adam
from OCR.document_OCR.v_attention.trainer_pg_va import Manager
from OCR.document_OCR.v_attention.models_pg_va import VerticalAttention, LineDecoderCTC
from basic.models import FCN_Encoder
from basic.generic_dataset_manager import OCRDataset
import torch
import torch.multiprocessing as mp


Apex not installed


In [2]:

%load_ext autoreload
%autoreload 2
from OCR.document_OCR.v_attention.parameters  import params # parameters
#from OCR.document_OCR.v_attention.models_pg_va import VerticalAttention, LineDecoderCTC


1


In [3]:
def test(rank, params):
    params["training_params"]["ddp_rank"] = rank
    model = Manager(params)
    # Model trains until max_time_training or max_nb_epochs is reached
    #model.train()

    # load weights giving best CER on valid set
    model.params["training_params"]["load_epoch"] = "best"
    model.load_model()


    # compute metrics on train, valid and test sets (in eval conditions)
    metrics = ["cer", "wer", "diff_len", "time", "worst_cer"]
    for dataset_name in params["dataset_params"]["datasets"].keys():
        for set_name in ["test", "valid", "train"]:
            model.predict("{}-{}".format(dataset_name, set_name), [(dataset_name, set_name), ], metrics, output=True)



In [4]:
if __name__ == "__main__":

    dataset_name = "IAM"  # ["RIMES", "IAM", "READ_2016"]
    # /media/aniketag/c4eb0693-4a65-4f0c-8d65-a6dad4b97ff9/IAM/formatted
    #

    if params["training_params"]["stop_mode"] == "learned":
        params["training_params"]["train_metrics"].append("loss_ce")
    params["model_params"]["stop_mode"] = params["training_params"]["stop_mode"]
    
    if params["training_params"]["use_ddp"] and not params["training_params"]["force_cpu"]:
        mp.spawn(test, args=(params,), nprocs=params["training_params"]["nb_gpu"])
    else:
        test(0, params)


##################
Available GPUS: 1
Rank 0: Quadro RTX 8000 _CudaDeviceProperties(name='Quadro RTX 8000', major=7, minor=5, total_memory=48598MB, multi_processor_count=72)
##################
Local GPU:
Rank 0: Quadro RTX 8000 _CudaDeviceProperties(name='Quadro RTX 8000', major=7, minor=5, total_memory=48598MB, multi_processor_count=72)
##################

	 stamp 1

	 datasets[key] = /media/aniketag/c4eb0693-4a65-4f0c-8d65-a6dad4b97ff9/IAM/formatted/IAM_paragraph/

	 joinPath = /media/aniketag/c4eb0693-4a65-4f0c-8d65-a6dad4b97ff9/IAM/formatted/IAM_paragraph/labels.pkl

	 is file: True

	 stamp 1.1

	 stamp 1.3

	 paths_and_sets: [{'path': '/media/aniketag/c4eb0693-4a65-4f0c-8d65-a6dad4b97ff9/IAM/formatted/IAM_paragraph', 'set_name': 'train'}]

	 from_segmentation: False  	 paths_and_sets: [{'path': '/media/aniketag/c4eb0693-4a65-4f0c-8d65-a6dad4b97ff9/IAM/formatted/IAM_paragraph', 'set_name': 'train'}]

	 samples # of keys:: 16

	 stamp 1.3.1

	 stamp 1.4.1 [239.41648798, 239.41648798

Prediction:   0%|                                                                                                                                                                       | 0/8 [00:00<?, ?it/s]/home/aniketag/anaconda3/envs/tor11/lib/python3.7/site-packages/torch/nn/functional.py:3463: UserWarning: Default upsampling behavior when mode=bicubic is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
Prediction:   0%|                                                                                                                                                                       | 0/8 [00:00<?, ?it/s]


	 ind_batch: 0 	 batch_data.keys(): dict_keys(['names', 'ids', 'nb_lines', 'labels', 'raw_labels', 'unchanged_labels', 'labels_len', 'imgs', 'imgs_shape', 'imgs_reduced_shape', 'line_raw', 'line_labels', 'line_labels_len']) 
 name: ['IAM_paragraph/test/test_6.png']

	 inside evaluate_batch!!

	 batch_data.keys(): dict_keys(['names', 'ids', 'nb_lines', 'labels', 'raw_labels', 'unchanged_labels', 'labels_len', 'imgs', 'imgs_shape', 'imgs_reduced_shape', 'line_raw', 'line_labels', 'line_labels_len'])

	 input image shape x = torch.Size([1, 3, 786, 925]) 	 orgH: 786 	 orgW: 925

	 x_reduced_len = 1

	 features.shape: torch.Size([1, 256, 25, 116])

	 h= 25 	 w: 116 	 torch.__version__: 1.8.0

	 attention_weights = torch.Size([1, 25])

	 max_nb_lines: 30

	 coverage = torch.Size([1, 25])

	 hidden = torch.Size([1, 1, 256]) 	 len: 2

	 imgName = IAM_paragraph/test/test_6.png

	 imgName =>>> test_6

	 calling attention!!: 0

	 inside verticle attention!!!!

	 1. features.size(): torch.Size([1

RuntimeError: Tensor for argument #3 'mat2' is on CPU, but expected it to be on GPU (while checking arguments for addmm)

In [ ]:
import numpy as np

attention_weights1 = torch.randn([2, 761, 949])
attention_weights1 = attention_weights1.unsqueeze(1)

x = torch.randn([2,3,949,761])
print("\n\t 1. attention_weights1 =",attention_weights1.shape," \t image.shape:",x.shape)

#x = x.cpu().numpy()
#attention_weights1 = attention_weights1.cpu().numpy().transpose(0,1, 3,2)
 
attention_weights1 = attention_weights1.cpu().numpy().transpose(0,3,2,1)
attention_weights1 = np.concatenate((attention_weights1,attention_weights1,attention_weights1),axis=3)

print("\n\t 2. attention_weights1 =",attention_weights1.shape," \t image.shape:",x.shape)

x = x.cpu().numpy().transpose(0, 2, 3,1) 
x = x + attention_weights1

print("\n\t 2. attention_weights1 =",attention_weights1.shape," \t image.shape:",x.shape)



In [ ]:
batch_size = 2
height = 500
width = 10


attention_weights1 = torch.randn(batch_size,height)
new_tensor = attention_weights1.repeat(1,width)

print(new_tensor.shape)

new_tensor = new_tensor.reshape(batch_size,height,-1)

print(new_tensor.shape)

In [ ]:
import torch
from torchvision.utils import save_image

attention_weights1 = torch.randn(2, 3, 480, 910)

for i in range(attention_weights1.shape[0]):
    # Extract the i-th image from the tensor
    image_i = attention_weights1[i]
    print("\n\t image_i =",image_i.shape)    
    # Save the image as a PNG file
    save_image(image_i, f'image_{i}.png')
